In [0]:
import torch
import time
from torchvision import datasets, transforms
import torch.nn as nn        
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import Image, display

In [0]:
class Hyperparameters():
    def __init__(self, train_bs, test_bs, lr, momentum, epochs):
        self.train_bs = train_bs
        self.test_bs = test_bs
        self.lr = lr
        self.momentum = momentum
        self.epochs = epochs

In [0]:
# defines linear classifier model 
# 400 -> 10 Linear Function -> log_softmax
class LinearModel(nn.Module):
    hyperparameters = Hyperparameters(100, 100, 0.01, 0.5, 5)
    def __init__(self):
        # define layers of net
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(400, 10)
        
        
    def forward(self, x):
        # define the forward prop function
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)
    
    def preprocess(self, data, bs):
        # preprocess input data
        return data.view(bs, 400)

In [0]:
class HiddenModel(nn.Module):
    hyperparameters = Hyperparameters(100, 100, 0.01, 0.6, 10)
    def __init__(self):
        super(HiddenModel, self).__init__()
        self.fc1 = nn.Linear(784, 800)
        self.fc2 = nn.Linear(800, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x, dim=1)
    
    def preprocess(self, data, bs):
        # preprocess input data
        return data.view(bs, 784)

In [0]:
MODELS = {
    'Linear': LinearModel,
    'Hidden': HiddenModel
}

In [0]:
# returns training data in tuple where 
# tuple[0] = X =  [1,20,20] tensor (20x20 input image)
# tuple[1] = Y = [] tensor (Scalar output value)
def get_train_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    return datasets.MNIST('./data', 
                          train=True, 
                          transform=transform, 
                          target_transform=None, 
                          download=True)

In [0]:
# returns data batched according to specified batch size
def batch_data(data, batch_size):
    return torch.utils.data.DataLoader(
        data, 
        pin_memory=True,
        batch_size=batch_size, 
        shuffle=True)
    

In [0]:
# returns testing data in tuple where 
# tuple[0] = X =  [1,20,20] tensor (20x20 input image)
# tuple[1] = Y = [] tensor (Scalar output value)
def get_test_data():
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    return datasets.MNIST('./data',
                          train=False,
                          transform=transform,
                          target_transform=None,
                          download=True)

In [0]:
def train(model, device, train_batches, optimizer, epoch):
    model.train() # put model in training mode
    
    for batch_idx, (data, target) in enumerate(train_batches):
        batch_size = len(data) # Question: Is this weird????
        data = model.preprocess(data, batch_size) # reshape data to be 1 * 400
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # negative log likelihood loss
        loss.backward()
        optimizer.step()
                
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, len(train_batches.dataset),
                100. * batch_idx / len(train_batches), loss.item()))

In [0]:
def test(model, device, test_batches):
    #print ("--- Testing ---")
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_batches):
            batch_size = len(data)
            data = model.preprocess(data, batch_size)
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_batches.dataset)

    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_batches.dataset),
        100. * correct / len(test_batches.dataset)))

In [0]:
def main():
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda") if use_cuda else torch.device("cpu")
    model_name = 'Hidden'
    model = MODELS[model_name]()
    
    hyperparams = model.hyperparameters
    train_bs = hyperparams.train_bs # training batch size
    test_bs = hyperparams.test_bs # test batch size
    lr = hyperparams.lr # learning rate
    momentum = hyperparams.momentum # momentum ??
    epochs = hyperparams.epochs # epochs
    
    
    train_data = get_train_data()
    train_batches = batch_data(train_data, train_bs)
    
    test_data = get_test_data()
    test_batches = batch_data(test_data, test_bs)
    
    
    model = model.to(device) # load model to cpu
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    start = time.perf_counter()
    for epoch in range(1, epochs + 1):
        train(model, device, train_batches, optimizer, epoch)
        test(model, device, test_batches)
    end = time.perf_counter()
    print('Done!: ' + str(end-start))

    #torch.save(model, './linearclassifier.pth')
    

In [57]:
main()

Done!: 90.48243811599991


In [0]:
get_train_data()[0]

torch.Size([1, 28, 28])

In [5]:
torch.cuda.is_available()


True